# Building Your First Campaign

This notebook will walk you through the steps to build a recommendation model for Allstore based on data collected from Amazon Outdoor equipment dataset. The goal is to recommend Amazon outdoor products that are relevant based on a particular user.

Note: You can bring you own dataset, but you need to ensure the schema of your dataset matches the intructions in this notebook.

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
# Configure the SDK to Personalize:

### Configure the SDK to Personalize:
Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [2]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

#### Create Personalize Role

Also Amazon Personalize needs the ability to assume Roles in AWS in order to have the permissions to execute certain tasks, the lines below grant that.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Configure the data

Data is imported into Amazon Personalize through Amazon S3, below we will specify a bucket that you have created within AWS for the purposes of this exercise.

Below you will update the `bucket` variable to instead be set to the value that you created earlier in the CloudFormation steps, this should be in a text file from your earlier work. the `filename` does not need to be changed.

To adapt this to production use, there should be a "staging" S3 bucket for raw data processing prior to the transfer of compatible data to a bucket where Personalize will use to import the data

# ⚠️ Specify a Bucket and LambdaARN

In [59]:
bucket = "teststr-staging"       # replace with the name of your S3 bucket
interactions_filename = "useritems-interactions.csv"
items_filename = "items_w_metadata.csv"

Rerank_arn = 'arn:aws:lambda:us-east-1:962222257213:function:teststr-Rerank'
GetRecommendationsByItem_arn = 'arn:aws:lambda:us-east-1:962222257213:function:teststr-GetRecommendationsByItem'
GetRecommendations_arn = 'arn:aws:lambda:us-east-1:962222257213:function:teststr-GetRecommendations'
PostClickEvent_arn = 'arn:aws:lambda:us-east-1:962222257213:function:teststr-PostClickEvent'

Prepare, and Upload Training Data

At present you do not have the Allstore data loaded locally yet for examination, execute the lines below to upload the Dataset to your staging S3 bucket

In [10]:
!echo "Bucket Name : $bucket"
!echo "uploading Dataset to Amazon s3"
!aws s3 cp ./Datasets s3://$bucket/ --recursive --exclude ".*"

Bucket Name : teststr-staging
uploading Dataset to Amazon s3
upload: Datasets/users.csv to s3://teststr-staging/users.csv     
upload: Datasets/items_w_metadata.csv to s3://teststr-staging/items_w_metadata.csv
upload: Datasets/useritems-interactions.csv to s3://teststr-staging/useritems-interactions.csv


#### Explore the Dataset

Users-Items Interaction dataset

In [15]:
ratinguri = f's3://{bucket}/useritems-interactions.csv'
useritems = pd.read_csv(ratinguri, names=['USER_ID', 'ITEM_ID', 'TIMESTAMP'], header=1)
pd.set_option('display.max_rows', 4) #change the bucket name to your bucket name

useritems

,USER_ID,ITEM_ID,TIMESTAMP
0,A1L5P841VIO02V,1881509818,1328140800
1,AB2W04NI4OEAD,1881509818,1330387200
...,...,...,...
253014,APRNS6DB68LLV,B00LFPS0CY,1405900800
253015,A3UJRNI8UR4871,B00LFPS0CY,1405382400


Items and metadata dataset

In [16]:
ratinguri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(ratinguri, names=['asin', 'title', 'img_url','genre'], header=1)
pd.set_option('display.max_rows', 4) #change the bucket name to your bucket name

items

,asin,title,img_url,genre
0,2094869245,5 LED Bicycle Rear Tail Red Bike Torch Laser B...,http://ecx.images-amazon.com/images/I/51RtwnJw...,Transportation|Vehicle|Bumper|Light|Wheel|Mach...
1,7245456313,Black Mountain Products Resistance Band Set wi...,http://ecx.images-amazon.com/images/I/51FdHlZS...,Cable|Electronics|Computer|Adapter
...,...,...,...,...
18344,B00LA12PNI,[2 PACK] Premium Dry Bags - two 500D PVC Tarp ...,http://ecx.images-amazon.com/images/I/518UdYxF...,Apparel|Lifejacket|Clothing|Vest|Bag
18345,B00LFPS0CY,Hydracentials Sporty 25 Oz Insulated Stainless...,http://ecx.images-amazon.com/images/I/41oC4FCS...,Shaker|Bottle|Water Bottle


user dataset

In [17]:
ratinguri = f's3://{bucket}/users.csv'
users = pd.read_csv(ratinguri, names=['userid'], header=1)
pd.set_option('display.max_rows', 4) #change the bucket name to your bucket name

users

,userid
0,A1L5P841VIO02V
1,AB2W04NI4OEAD
...,...
177766,AKPYS9VSM3LNX
177767,AXBNEFRD90GLM


### Create Schema

A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [18]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-usersitems-interactions",
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:962222257213:schema/personalize-demo-usersitems-interactions",
  "ResponseMetadata": {
    "RequestId": "0f032ccd-259d-4527-8989-cc851646284e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 13:34:23 GMT",
      "x-amzn-requestid": "0f032ccd-259d-4527-8989-cc851646284e",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [19]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TITLE",
            "type": "string"
        },
        {
            "name": "IMGURL",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-items",
    schema = json.dumps(items_schema)
)

items_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:962222257213:schema/personalize-demo-items",
  "ResponseMetadata": {
    "RequestId": "916af791-493f-46f7-a41f-6ed9f38e71e4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 13:34:24 GMT",
      "x-amzn-requestid": "916af791-493f-46f7-a41f-6ed9f38e71e4",
      "content-length": "88",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Create and Wait for Dataset Group

The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like.

#### Create Dataset Group

In [20]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-launch-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:962222257213:dataset-group/personalize-launch-demo",
  "ResponseMetadata": {
    "RequestId": "7c181529-ffb2-45cf-a42f-c6bc4a9f60f3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 13:34:26 GMT",
      "x-amzn-requestid": "7c181529-ffb2-45cf-a42f-c6bc4a9f60f3",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Group to Have ACTIVE Status

Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [21]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


#### Create Dataset User-Item Interactions Dataset

After the group, the next thing to create is the actual datasets, in this example we will only create 1 for the interactions data. Execute the cells below to create it.

In [22]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:962222257213:dataset/personalize-launch-demo/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "defe64f6-bbe5-49a6-a8e5-5a19af778e15",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 13:35:28 GMT",
      "x-amzn-requestid": "defe64f6-bbe5-49a6-a8e5-5a19af778e15",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [23]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:962222257213:dataset/personalize-launch-demo/ITEMS",
  "ResponseMetadata": {
    "RequestId": "74858393-396c-4b4e-a077-dd40d8603237",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 13:35:28 GMT",
      "x-amzn-requestid": "74858393-396c-4b4e-a077-dd40d8603237",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Attach Policy to S3 Bucket

Amazon Personalize needs to be able to read the content of your S3 bucket that you created earlier. The lines below will do that.

In [24]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '94CE7697AC878BC3',
  'HostId': '/kcldRz6bDNZ0iLzFxQTuAYnnVykCVvtUMJlsc3bPUM8x0OxPMbI3ex7SHyK9VECcshlr0OJcgI=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '/kcldRz6bDNZ0iLzFxQTuAYnnVykCVvtUMJlsc3bPUM8x0OxPMbI3ex7SHyK9VECcshlr0OJcgI=',
   'x-amz-request-id': '94CE7697AC878BC3',
   'date': 'Mon, 24 Feb 2020 13:35:30 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Import the data

Earlier you created the DatasetGroup and Dataset to house your information, now you will execute an import job that will load the data from S3 into Amazon Personalize for usage building your model.

#### Create Dataset Import Job Interactions

In [25]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:962222257213:dataset-import-job/personalize-demo-import-interactions",
  "ResponseMetadata": {
    "RequestId": "a1247271-bbee-4b3c-bac8-8233838a8524",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 13:35:29 GMT",
      "x-amzn-requestid": "a1247271-bbee-4b3c-bac8-8233838a8524",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Import Job to Have ACTIVE Status

It can take a while before the import job completes, please wait until you see that it is active below.

In [38]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


#### Create Dataset Import Job Items

In [39]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

items_interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateDatasetImportJob operation: Another resource with Arn arn:aws:personalize:us-east-1:962222257213:dataset-import-job/personalize-demo-import-items already exists.

#### Wait for Dataset Import Job to Have ACTIVE Status

It can take a while before the import job completes, please wait until you see that it is active below.

In [40]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [41]:
list_recipes_response = personalize.list_recipes()
hrnn_arn = list_recipes_response['recipes'][0]['recipeArn']
rerank_arn = list_recipes_response['recipes'][3]['recipeArn']
sims_arn = list_recipes_response['recipes'][5]['recipeArn']
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

Here you can see below that we are picking the `HRNN` recipie.

# HRNN Solution

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [42]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_arn
)

hrnn_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateSolution operation: Another resource with Arn arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-hrnn already exists.

#### Create Solution Version

In [43]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-hrnn/e342ae42",
  "ResponseMetadata": {
    "RequestId": "52b7f5d0-ce28-4794-a099-88e19d50a064",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 24 Feb 2020 16:49:02 GMT",
      "x-amzn-requestid": "52b7f5d0-ce28-4794-a099-88e19d50a064",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [47]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: ACTIVE


#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [48]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-hrnn/e342ae42",
  "metrics": {
    "coverage": 0.0643,
    "mean_reciprocal_rank_at_25": 0.0091,
    "normalized_discounted_cumulative_gain_at_10": 0.0115,
    "normalized_discounted_cumulative_gain_at_25": 0.0139,
    "normalized_discounted_cumulative_gain_at_5": 0.0104,
    "precision_at_10": 0.0016,
    "precision_at_25": 0.0011,
    "precision_at_5": 0.0025
  },
  "ResponseMetadata": {
    "RequestId": "1d22b794-3e6a-47db-887a-e5bd3c9aa61b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:07:54 GMT",
      "x-amzn-requestid": "1d22b794-3e6a-47db-887a-e5bd3c9aa61b",
      "content-length": "410",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [49]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-camp-hrnn",
    solutionVersionArn = hrnn_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateCampaign operation: Another resource with Arn arn:aws:personalize:us-east-1:962222257213:campaign/personalize-camp-hrnn already exists.

#### Wait for Campaign to Have ACTIVE Status

In [50]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = hrnn_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: ACTIVE


## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. Then we will create a few helper functions for getting the Allstore merchandise (item descriptions) information to show for recommendations instead of just IDs.

In [51]:
# Getting a random user:
user_id, item_id, _ = useritems.sample().values[0]
print("USER: {}".format(user_id))

USER: A2AM9IPXQKNO8L


In [54]:
# First load items into memory
allitemuri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(allitemuri, sep=',', usecols=[0,1], names=['asin', 'title'], index_col='asin',header=0)

# print(items)

def get_allstore_products(asin):
    """
    Takes in an ID, returns a title
    """
    asin = str(asin)
    return items.loc[asin]['title']

#### Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of movies that are recommended.


In [55]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    try :
        title = get_allstore_products(item['itemId'])
    except:
        continue
    recommendation_list.append(title)  
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Item Description'])
recommendations_df

Recommendations for user:  A2AM9IPXQKNO8L


,Item Description
0,Emergency Mylar Thermal Blankets (Pack of 10)
1,Park Tool Vulcanizing Patch Kit - VP-1
2,Potable Aqua Water Treatment Tablets
3,Petzl E91 Tikkina 2 Headlamp
4,GSI Outdoors Glacier Stainless Bottle Cup/Pot
5,Gerber 31-000699 Bear Grylls Survival Series F...
6,ALPS Mountaineering Compression Sleeping Bag S...
7,Eagles Nest Outfitters DryFly Raintarp
8,Paracord Planet Type III 7 Strand 550 Paracord...
9,Magnesium Fire Starter


## Review

Using the codes above you have successfully trained a deep learning model to generate Allstore Merchandise recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `2.Building_Campaign_SIMS.ipynb`



## Update Lambda Configuration

In [56]:
lambdaclient = boto3.client('lambda')

response = lambdaclient.get_function(
    FunctionName=GetRecommendations_arn,
)

Environment = response['Configuration']['Environment']
print(Environment)

#update Env var

Environment['Variables']['Campaign_ARN'] = hrnn_campaign_arn

response = lambdaclient.update_function_configuration(
    FunctionName=GetRecommendations_arn,
    Environment=Environment
)

{'Variables': {'Campaign_ARN': 'arn:aws:personalize:us-east-1:387269085412:campaign/personalize-demo-camp', 'ddb_tablename': 'teststr-Items'}}


## Notes for the Next Notebook:

There are a few values you will need for the next notebook, execute the cells below to store them so they can be copied and pasted into the next part of the exercise.

In [60]:
%store Rerank_arn
%store GetRecommendationsByItem_arn
%store PostClickEvent_arn
%store dataset_group_arn
%store hrnn_campaign_arn
%store interactions_dataset_arn
%store items_dataset_arn
%store useritems
%store recommendations_df
%store bucket
%store user_id


Stored 'Rerank_arn' (str)
Stored 'GetRecommendationsByItem_arn' (str)
Stored 'PostClickEvent_arn' (str)
Stored 'dataset_group_arn' (str)
Stored 'hrnn_campaign_arn' (str)
Stored 'interactions_dataset_arn' (str)
Stored 'items_dataset_arn' (str)
Stored 'useritems' (DataFrame)
Stored 'recommendations_df' (DataFrame)
Stored 'bucket' (str)
Stored 'user_id' (str)
